In [11]:
import sys
sys.path.append('..')

import math
import matplotlib.pyplot as plt
import numpy as np
import scipy

import metrics

Root findings algorithms are algorithms to compute the roots of a continuous function.

# Real functions

Let $f: \mathbb{R} \to \mathbb{R}$.  
The goal is to find $x \in \mathbb{R}$ such that $f(x) =0$

## Bisection method

The bisection method start with 2 number $a$ and $b$ such that $a < b$ and $f(a)$ is of opposed sign to $f(b)$.

It computes the midpoint $c$:
$$c = \frac{a+b}{2}$$

then 3 possibilites:
- $f(c) \approx 0$: $c$ is a root
- $f(c)$ is of same sign than $f(a)$: keep searching with $[c,b]$
- $f(c)$ is of same sign than $f(b)$: keep searching with $[a,c]$

In [50]:
import scipy.stats

def root_bisection(f, a, b):
    MAX_ITERS = 500
    TOL = 1e-25
    
    for i in range(MAX_ITERS):
        
        c = (a + b) / 2
        fc = f(c)
        if fc**2 < TOL:
            break
            
        if np.sign(fc) == np.sign(f(a)):
            a = c
        else:
            b = c
        
    return c


def norm_cdf(x):
    return 1/2 * (1 + scipy.special.erf(x / np.sqrt(2)))

def norm_quantile(x):
    def f(v):
        return norm_cdf(v) - x
    return root_bisection(f, -10, 10)

def randn_qt(size):
    u = np.random.rand(size)
    x = np.array([norm_quantile(v) for v in u])
    return x

v = np.linspace(1e-3, 1-1e-3, 1000)
b1 = scipy.stats.norm.ppf(v)
b2 = [norm_quantile(x) for x in v]
print(b1[::100])
print(b2[::100])
print(metrics.tdist(b1, b2))

[-3.09023231e+00 -1.27644063e+00 -8.38767842e-01 -5.22389164e-01
 -2.51795418e-01  1.25205990e-03  2.54381035e-01  5.25261523e-01
  8.42332969e-01  1.28211644e+00]
[-3.090232306130929, -1.2764406343944756, -0.8387678422332101, -0.5223891635955624, -0.2517954179336357, 0.0012520598954779416, 0.254381034810649, 0.5252615229744606, 0.8423329692368497, 1.282116442584993]
8.036040496073416e-11


## Secant Method

Start with initial values $x_0$ and $x_1$, and define $y_0=f(x_0)$, $y_1=f(x_1)$

We find the line passing by $(x_0,y_0)$, and $(x_1,y_1)$:
$$y = ax + b$$
$$a = \frac{y_1 - y_0}{x_1 - x_0}$$
$$b = y_1 - ax_1$$

This simplifies at:
$$y = a(x - x_1) + y_1$$

Solving for $y=0$, we get:
$$x = x_1 - y_1 * \frac{1}{a}$$

We define the next point $x_2$ as:

$$x_2 = x_1 - f(x_1) \frac{x_1 - x_0}{f(x_1) - f(x_0)}$$  

$x_2$ is a closer approximation than $x_0$ and $x_1$ to $f(x) = 0$.  
We repeat the process iteratively. $x_n$ converges to the solution.

In [64]:

def root_secant(f, x0, x1):
    
    MAX_ITERS = 500
    TOL = 1e-25
    
    for i in range(MAX_ITERS):
        y0 = f(x0)
        y1 = f(x1)
        if y1**2 < TOL:
            break
            
        #print('f({:.3f})={:.3f}, f({:.3f})={:.3f}'.format(x0, y0, x1, y1))    
            
            
        x2 = x1 - y1 * (x1-x0)/(y1-y0) 
        
        x0 = x1
        x1 = x2
            
    return x1
    

def norm_cdf(x):
    return 1/2 * (1 + scipy.special.erf(x / np.sqrt(2)))

def norm_quantile(x):
    def f(v):
        return norm_cdf(v) - x
    return root_secant(f, -1, 1)

def randn_qt(size):
    u = np.random.rand(size)
    x = np.array([norm_quantile(v) for v in u])
    return x

v = np.linspace(1e-3, 1-1e-3, 1000)
b1 = scipy.stats.norm.ppf(v)
b2 = [norm_quantile(x) for x in v]
print(b1[::100])
print(b2[::100])
print(metrics.tdist(b1, b2))

[-3.09023231e+00 -1.27644063e+00 -8.38767842e-01 -5.22389164e-01
 -2.51795418e-01  1.25205990e-03  2.54381035e-01  5.25261523e-01
  8.42332969e-01  1.28211644e+00]
[-3.090232306167058, -1.2764406343940464, -0.8387678422325504, -0.522389163595099, -0.2517954179337919, 0.001252059895742148, 0.2543810348108902, 0.5252615229742582, 0.8423329692359971, 1.282116442586217]
1.5835924257365702e-11
